In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Define dataset path
dataset_path = "fruits_dataset"  

# Image parameters
IMG_SIZE = (224, 224)  # Standard size for CNN models
BATCH_SIZE = 32  # Adjust based on available RAM

# Data Augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values
    rotation_range=20,  
    width_shift_range=0.2,  
    height_shift_range=0.2,  
    shear_range=0.2,  
    zoom_range=0.2,  
    horizontal_flip=True,  
    validation_split=0.2  # 20% of data for validation
)

# Load training data
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",  # Multiclass classification
    subset="training"
)

# Load validation data
val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)

# Print class labels
print("Class labels:", train_generator.class_indices)


Found 1325 images belonging to 6 classes.
Found 330 images belonging to 6 classes.
Class labels: {'fresh_peaches_done': 0, 'fresh_pomegranates_done': 1, 'fresh_strawberries_done': 2, 'rotten_peaches_done': 3, 'rotten_pomegranates_done': 4, 'rotten_strawberries_done': 5}


In [2]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam


In [4]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Load MobileNetV2 (pretrained on ImageNet) without top layers
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
base_model.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Reduce feature dimensions
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
output_layer = Dense(len(train_generator.class_indices), activation="softmax")(x)  # Corrected

# Define final model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Summary
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,422,726 (9.24 MB)

 Trainable params: 164,742 (643.52 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [5]:
# Train the model
EPOCHS = 10  # Increase for better accuracy

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS
)


C:\Users\VINIT\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 40s 893ms/step - accuracy: 0.5476 - loss: 1.1746 - val_accuracy: 0.8152 - val_loss: 0.4716
Epoch 2/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 15s 368ms/step - accuracy: 0.8559 - loss: 0.3865 - val_accuracy: 0.8606 - val_loss: 0.3531
Epoch 3/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 16s 372ms/step - accuracy: 0.8842 - loss: 0.3243 - val_accuracy: 0.8545 - val_loss: 0.3411
Epoch 4/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 16s 373ms/step - accuracy: 0.8967 - loss: 0.2747 - val_accuracy: 0.8606 - val_loss: 0.4181
Epoch 5/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 16s 372ms/step - accuracy: 0.9063 - loss: 0.2374 - val_accuracy: 0.8939 - val_loss: 0.3155
Epoch 6/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 16s 379ms/step - accuracy: 0.9215 - loss: 0.2011 - val_accuracy: 0.8848 - val_loss: 0.3142
Epoch 7/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 25s 602ms/step - accuracy: 0.9238 - loss: 0.1904 - val_accuracy: 0.8667 - val_loss: 0.3649
Epoch 8/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 16s 384ms/step - accuracy: 0.9401 - loss: 0.1741 - val_accu

In [6]:
# Save the trained model
model.save("fruit_classifier.h5")
print("Model saved successfully!")


Model saved successfully!


In [7]:
from tensorflow.keras.models import load_model
import numpy as np
import cv2

# Load the trained model
model = load_model("fruit_classifier.h5")

# Print class labels
class_labels = list(train_generator.class_indices.keys())
print("Class Labels:", class_labels)


Class Labels: ['fresh_peaches_done', 'fresh_pomegranates_done', 'fresh_strawberries_done', 'rotten_peaches_done', 'rotten_pomegranates_done', 'rotten_strawberries_done']


In [9]:
def predict_image(image_path):
    # Load image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))  # Resize to model input size
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    # Get prediction
    prediction = model.predict(img)
    class_index = np.argmax(prediction)  # Get highest probability class
    confidence = np.max(prediction)  # Get confidence score

    # Return result
    return class_labels[class_index], confidence

# Example usage
image_path = "rotten_peach_43.jpg"  # Change this to your test image
predicted_class, confidence = predict_image(image_path)
print(f"Predicted: {predicted_class} with {confidence:.2f} confidence")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Predicted: rotten_peaches_done with 0.78 confidence
